# Content-based Filtering

The content-based filtering recommendation algorithm suggests items similar to ones the user has liked before by analyzing the features of the items and the user’s preferences. It compares the content (e.g., product descriptions, keywords, tags) of items with a user’s historical interactions (such as movies they have watched or articles they have read) and suggests items with similar attributes.

Imagine a user watching a sci-fi movie, and the system extracts features like “genre: sci-fi,” “director: Christopher Nolan,” and “themes: space exploration.” The system would then recommend other movies sharing these features, e.g., another sci-fi film with similar themes and directors.

In this notebook, we would take a very simple and straight-forward approach to implement Content-based Filtering. We make recommendations by finding the most similar items to the user's last interacted item, where similarity is defined based on the textual description of the items. The score is the similarity between the target item and the neighbors.

Specifically, we will:
- Select a few description fields from the item metadata
- Transform the features into numerical format, e.g. TFIDF for text data, Multi-Hot Encoders for categorical features
- Concat and normalize the transformation output
- Use cosine similarity to look up neighbor items and use it as the prediction score

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import dill
import mlflow
import numpy as np  # required for the scikit-learn pipeline to work
import pandas as pd
import torch
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from sklearn.preprocessing import MinMaxScaler

load_dotenv()

sys.path.insert(0, "..")


from src.eval import (
    create_label_df,
    create_rec_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.id_mapper import IDMapper
from src.model import ContentBased
from src.train_utils import map_indice

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L4 - Reco Algo"
    run_name: str = "005-content-based"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-26 22:59:29.990 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L4 - Reco Algo - run 005-content-based...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L4 - Reco Algo",
  "run_name": "005-content-based",
  "notebook_persist_dp": "/Users/dvq/frostmourne/reco-algo/notebooks/data/005-content-based",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Implement

In [4]:
def init_model(train_item_features, device):
    model = ContentBased(item_features=train_item_features, device=device)
    return model

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
# device = 'cpu'
logger.info(f"Using {device} device")

2024-09-26 22:59:30.158 | INFO     | __main__:<module>:7 - Using mps device


In [6]:
with open("../data/item_metadata_pipeline.dill", "rb") as f:
    item_metadata_pipeline = dill.load(f)

# Test implementation

In [7]:
# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
ratings = [1, 4, 5, 3, 2]
timestamps = [0, 1, 2, 3, 4]
main_category = [
    "All Electronics",
    "Video Games",
    "All Electronics",
    "Video Games",
    "Unknown",
]
title = ["All Electronics", "Video Games", "All Electronics", "Video Games", "Unknown"]
description = [[], [], ["Video games blah blah"], [], ["blah blah"]]
categories = [[], ["Headsets"], ["Video Games"], [], ["blah blah"]]
price = ["from 14.99", "14.99", "price: 9.99", "20 dollars", "None"]

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.rating_col: ratings,
        args.timestamp_col: timestamps,
        "main_category": main_category,
        "title": title,
        "description": description,
        "categories": categories,
        "price": price,
    }
)
# Drop duplicated item features so that the ContentBased model will fit correctly in terms of index mapping
fit_df = train_df.drop_duplicates(subset=["item_indice"])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)

val_user_indices = [0, 1, 2]
val_item_indices = [2, 1, 2]
val_ratings = [2, 4, 5]
val_timestamps = [5, 6, 7]
val_main_category = ["All Electronics", "Video Games", "All Electronics"]
val_title = ["All Electronics", "Video Games", "All Electronics"]
val_description = [["Video games blah blah"], [], ["Video games blah blah"]]
val_categories = [["Video Games"], ["Headsets"], ["Video Games"]]
val_price = ["price: 9.99", "14.99", "price: 9.99"]

val_df = pd.DataFrame(
    {
        "user_indice": val_user_indices,
        "item_indice": val_item_indices,
        args.rating_col: val_ratings,
        args.timestamp_col: val_timestamps,
        "main_category": val_main_category,
        "title": val_title,
        "description": val_description,
        "categories": val_categories,
        "price": val_price,
    }
)
val_item_features = item_metadata_pipeline.transform(val_df).astype(np.float32)

In [8]:
n_users = len(set(user_indices))
n_items = len(set(item_indices))

model = init_model(train_item_features, device)

items1 = [1, 2]
items2 = [0, 3]
predictions = model.predict(items1, items2)
print(predictions)

print("\n\n")

users = [0, 1]
anchor_items = [2, 3]
recommendations = model.recommend(users, anchor_items, k=args.top_K)
print(recommendations)

tensor([0.1166, 0.0384], device='mps:0')





Generating Recommendations:   0%|          | 0/2 [00:00<?, ?it/s]

{'user_indice': [0, 0, 0, 0, 1, 1, 1, 1], 'recommendation': [0, 4, 1, 3, 1, 4, 0, 2], 'score': [0.6481854319572449, 0.13637012243270874, 0.03882099688053131, 0.03839520364999771, 0.9612818360328674, 0.3155585825443268, 0.16881369054317474, 0.03839520364999771]}


# Prep data

In [9]:
def get_last_item(df, item_sequence_col="item_sequence"):
    return df.assign(
        last_item_indice=lambda df: df[item_sequence_col].apply(lambda s: s[-1])
    )

In [10]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm = IDMapper().load("../data/idm.json")
assert (val_df[args.timestamp_col].min() - train_df[args.timestamp_col].max()) > 0
val_timestamp = train_df[args.timestamp_col].max() + 1
print(f"{val_timestamp=}")

val_timestamp=np.int64(1628641464793)


In [11]:
train_df = train_df.pipe(get_last_item)
val_df = val_df.pipe(get_last_item)
full_df = pd.concat([train_df, val_df], axis=0)
user_ids = train_df[args.user_col].values
item_ids = train_df[args.item_col].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-09-26 22:59:33.997 | INFO     | __main__:<module>:9 - len(unique_user_ids)=20,366, len(unique_item_ids)=4,696


In [12]:
train_df = train_df.pipe(map_indice, idm, args.user_col, args.item_col)
val_df = val_df.pipe(map_indice, idm, args.user_col, args.item_col)
full_df = full_df.pipe(map_indice, idm, args.user_col, args.item_col)

In [13]:
train_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice
0,AETXPRHIFDJPZH62JOVAKYPN4IJA,B07HC4ZD3B,0.0,1572818050904,13294,1565,Video Games,Poké Ball Plus Charge Stand Officially License...,[Create a Pokemon center at home with HORI's N...,"[Video Games, Nintendo Switch, Accessories, Ba...",189.98,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....",3092.0
1,AE4UCYLS3NXHINCFZW6FFPT7VX5A,B003S9WQAM,0.0,1289338003000,7769,1208,Video Games,EA Sports Active 2 - Playstation 3,"[Product Description, EA SPORTS Active 2 with ...","[Video Games, Legacy Systems, PlayStation Syst...",41.99,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....",-1.0
2,AHHRPUW2QJC7F2OGM3ACAHW4PXVQ,B00B1OYBBS,0.0,1420679625000,9498,4137,Video Games,Enemy Front - Xbox 360,[Against the visceral backdrop of breathtaking...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",None,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....",240.0
3,AF6GXCYIAWC6G33YPOJ5IBGD6RKQ,B004C43FH0,0.0,1427907775000,12279,989,Video Games,Rollercoaster Tycoon 2: Triple Thrill Pack [Do...,"[From the Manufacturer, This compilation of Ro...","[Video Games, PC, Games]",9.99,"[1042.0, 2148.0, 4637.0, 54.0, 2854.0, 4479.0,...",2663.0
4,AFAUFOADANHRMH3GR2FA73A6NHIA,B08VFQ3XJX,5.0,1408655680000,8779,472,Video Games,Final Fantasy X,"[Product Description, Final Fantasy X finally ...","[Video Games, Legacy Systems, PlayStation Syst...",20.0,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 3711.0, 4...",1122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
340085,AHNTZVONMCC55XXEESJJ2I3OMRMQ,B005IU8A5O,0.0,1445376835000,9511,2439,All Electronics,JINHEZO JINHEZO-CS-370 for Xbox 360 Kinect Sen...,[],"[Video Games, Legacy Systems, Xbox Systems, Xb...",7.99,"[-1.0, -1.0, -1.0, -1.0, -1.0, 1081.0, 115.0, ...",4663.0
340086,AFC4IA7LI5GB43LNPRFF37H2FYFQ,B00IAVDQCK,0.0,1483041968000,5491,2382,Video Games,Xbox One Stereo Headset,"[Surround your senses, Immerse yourself in ric...","[Video Games, Xbox One, Accessories, Headsets]",18.99,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 151...",1317.0
340087,AGJD6XPC33AX7OVG4GKFHLP5KMGA,B001EYUSC6,0.0,1489418014000,18182,1049,None,FIFA Soccer 09 - Sony PSP,"[Product description, Play professional soccer...","[Video Games, Legacy Systems, PlayStation Syst...",None,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 431...",4343.0
340088,AEFJKBRX2TPPMMJ53DKHANKRYDXQ,B08N6NMGNB,5.0,1510350967758,819,1720,Video Games,Thrustmaster T300 RS - Gran Turismo Edition Ra...,[Works with PS5 games (PS5 games compatibility...,"[Video Games, Legacy Systems, PlayStation Syst...",449.0,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 173...",586.0


# Fit

In [14]:
fit_df = train_df.drop_duplicates(subset=[args.item_col])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)
model = init_model(train_item_features, device)

# Predict

In [15]:
user_id = val_df.sample(1)[args.user_col].values[0]
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
test_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice
204,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B07CV6LH3V,4.0,1640980412806,16930,4018,Cell Phones & Accessories,"Mumba Case for Nintendo Switch, [Heavy Duty] S...",[],"[Video Games, Nintendo Switch, Accessories, Ca...",14.99,"[-1, -1, -1, -1, 60, 1529, 3749, 1057, 33, 3617]",3617
413,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B000KQQUMQ,0.0,1645373161344,16930,631,Video Games,The Sims 2 Seasons Expansion Pack - PC,[Experiencing all four seasons for the first t...,"[Video Games, PC, Games]",30.99,"[3749, 1057, 33, 3617, 4018, 2573, 4057, 1348,...",4492
462,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B07MWB5YJW,1.0,1644542532079,16930,4492,Video Games,HORI Game Card Case 24 for Nintendo Switch Off...,[Officially Licensed by Nintendo. Store & orga...,"[Video Games, Nintendo Switch, Accessories, Ca...",9.99,"[1529, 3749, 1057, 33, 3617, 4018, 2573, 4057,...",3994
559,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B09V25XG1G,5.0,1645373161344,16930,2163,Video Games,"8BitDo Pro 2 Bluetooth Controller for Switch, ...","[Compatible with Switch, Windows PC, macOS, An...","[Video Games, Nintendo Switch, Accessories, Co...",49.99,"[3749, 1057, 33, 3617, 4018, 2573, 4057, 1348,...",4492
646,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B0BNPKCKKT,0.0,1640981152142,16930,2454,Video Games,Nintendo Switch System Carrying Case – Protect...,[Protect and organize your Nintendo Switch Sys...,"[Video Games, Nintendo Switch, Accessories, Ca...",25.99,"[-1, 60, 1529, 3749, 1057, 33, 3617, 4018, 257...",4057
678,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B01EORDE48,0.0,1654817132480,16930,3128,Video Games,Attack on Titan - PlayStation 4,[Attack on Titan closely follows the riveting ...,"[Video Games, PlayStation 4, Games]",59.98,"[33, 3617, 4018, 2573, 4057, 1348, 3994, 4492,...",4184
974,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B07T85XHX6,5.0,1648688338983,16930,4184,Video Games,Ni no Kuni: Wrath of the White Witch - Nintend...,"[""Journey back to the other world in Ni no Kun...","[Video Games, Nintendo Switch, Games]",23.37,"[1057, 33, 3617, 4018, 2573, 4057, 1348, 3994,...",2163
1109,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B014XCWXIW,0.0,1640980805319,16930,4447,Video Games,Minecraft: Story Mode - Season Disc - Xbox One,"[As Jesse, you'll embark on a perilous adventu...","[Video Games, Xbox One, Games]",59.91,"[-1, -1, -1, 60, 1529, 3749, 1057, 33, 3617, 4...",4018
1116,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B01N3ASPNV,5.0,1640980885681,16930,4057,All Electronics,amFilm Tempered Glass Screen Protector for Nin...,[],"[Video Games, Nintendo Switch, Accessories, Fa...",8.91,"[-1, -1, 60, 1529, 3749, 1057, 33, 3617, 4018,...",2573
1337,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B08N7PZK77,5.0,1640980805319,16930,2573,Video Games,PowerA Enhanced Wireless Controller for Ninten...,[Play your favorite Nintendo Switch games like...,"[Video Games, Nintendo Switch, Accessories, Co...",43.49,"[-1, -1, -1, 60, 1529, 3749, 1057, 33, 3617, 4...",4018


In [16]:
item_id = test_df.loc[lambda df: df[args.rating_col].gt(0)][args.item_col].values[0]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
anchor_item_indice = test_df["last_item_indice"].values[0]
item_indice = idm.get_item_index(item_id)
model.predict([item_indice], [anchor_item_indice])

2024-09-26 22:59:37.117 | INFO     | __main__:<module>:2 - Test predicting before training with user_id = AG4RCXKPTC6QRORJLUSBY4SO2IAA and parent_asin = B07CV6LH3V


tensor([0.0138], device='mps:0')

# Recommend

In [17]:
val_anchor_item_indices = val_df["last_item_indice"].values
val_user_indices = val_df["user_indice"].values

In [18]:
recommendations = model.recommend(val_user_indices, val_anchor_item_indices, args.top_K)

Generating Recommendations:   0%|          | 0/1898 [00:00<?, ?it/s]

# Evaluate

## Ranking metrics

In [19]:
recommendations_df = pd.DataFrame(recommendations).pipe(
    create_rec_df, idm, args.user_col, args.item_col
)
recommendations_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,10525,2662,0.352492,1.0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B002I09286
1,10525,2868,0.344217,3.0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B001G6064C
2,10525,2693,0.327415,5.0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B073X4V4V4
3,10525,3938,0.309451,7.0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B002P5UBG6
4,10525,559,0.251890,9.0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B009CYJ8SA
...,...,...,...,...,...,...
189795,14213,685,0.274828,192.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B001EYUPX8
189796,14213,2585,0.274497,194.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B001EHD9BU
189797,14213,4614,0.272270,196.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B0000D9SQ6
189798,14213,862,0.270980,198.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B00008JODN


In [20]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

,user_id,parent_asin,rating,rating_rank
509,AFUQQLR2N2LY7XPE4VJ5YF3LDZVA,B07YN82X3B,5.0,1.0
250,AGJO7OFBOKRLDTSEL2HHSZSKDQ4Q,B07PZ8NZSZ,1.0,1.0
1549,AF4QBZD2EXOTKIOOH4BOC4HZDHYA,B08NRVRF3J,3.0,1.0
395,AH6YPZLRQH6OSGLGBVTGCNSF7JQQ,B0BVVTQ5JP,5.0,1.0
1325,AESEOKCWWKUG7YPP43J2CRWAXQIA,B09GM4283G,5.0,1.0
...,...,...,...,...
1365,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B00LMRL00O,0.0,18.0
528,AFB6FYPPCN33UMUU5536IHXNOHCQ,B00Z9TM72Q,0.0,18.0
1670,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B07NKN4VR4,0.0,18.0
311,AFB6FYPPCN33UMUU5536IHXNOHCQ,B002Z01QO2,0.0,19.0


In [21]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
179,16168.0,2484.0,0.611555,1,AE2AZ2MNROPF33U6SS53VI22OXJA,B07YBX6T9P,0,NaN
180,16168.0,2484.0,0.611555,2,AE2AZ2MNROPF33U6SS53VI22OXJA,B07YBX6T9P,0,NaN
50,16168.0,4175.0,0.573311,3,AE2AZ2MNROPF33U6SS53VI22OXJA,B001E53N36,0,NaN
51,16168.0,4175.0,0.573311,4,AE2AZ2MNROPF33U6SS53VI22OXJA,B001E53N36,0,NaN
113,16168.0,3527.0,0.552871,5,AE2AZ2MNROPF33U6SS53VI22OXJA,B00MP7SFJE,0,NaN
...,...,...,...,...,...,...,...,...
191497,1883.0,2806.0,0.174328,196,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B003JKK6FM,0,NaN
191597,1883.0,1875.0,0.173226,197,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B07G7YP27Q,0,NaN
191598,1883.0,1875.0,0.173226,198,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B07G7YP27Q,0,NaN
191556,1883.0,2123.0,0.172966,199,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B0191J0RXA,0,NaN


In [22]:
ranking_report = log_ranking_metrics(args, eval_df)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:61: RuntimeWarning: invalid value encountered in divide
  return (1 + beta_sqr) * precision_arr * recall_arr / (beta_sqr * precision_arr + recall_arr)


## Classification metrics

In [23]:
val_anchor_item_indices = val_df["last_item_indice"].values
val_item_indices = val_df["item_indice"].values

In [24]:
classifications = (
    model.predict(val_item_indices, val_anchor_item_indices)
    .cpu()
    .detach()
    .numpy()
    .reshape(-1, 1)
)
classifications = MinMaxScaler(feature_range=(0, 1)).fit_transform(classifications)

In [25]:
eval_classification_df = val_df.assign(
    classification_proba=classifications,
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice,classification_proba,label
0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B00V5V3E38,0.0,1633967679730,10525,4119,Video Games,Legend of Kay Anniversary - Nintendo Wii U,"[10 years after its initial release, Legend of...","[Video Games, Legacy Systems, Nintendo Systems...",None,"[-1, -1, -1, -1, 3766, 4102, 2872, 2810, 502, ...",536,0.140156,0
1,AGFPKXT34G5FGWARKXZC4GJTJQUQ,B01GY3651O,0.0,1630491281842,11724,2977,Video Games,XCOM 2 Deluxe Edition [Online Game Code],[The XCOM 2 Digital Deluxe Edition includes th...,"[Video Games, PC, Games]",None,"[594, 4578, 3289, 4664, 1057, 3845, 3010, 2691...",4567,0.135370,0
2,AH4AOFTTDPHPAFAAVFMAF25H2LIQ,B09B14PJCG,0.0,1641748747823,7401,4682,Video Games,A Plague Tale: Innocence (XB1) - Xbox One,[Follow the grim tale of young Amicia and her ...,"[Video Games, Xbox One, Games]",59.99,"[4064, 24, 1441, 1805, 2619, 2254, 2274, 1454,...",388,0.235695,0
3,AEXTTZIJDNXIXQZFR5O7IJRXO3GA,B081243BT6,0.0,1637074638494,5074,3104,Cell Phones & Accessories,Orzly Carrying case for Nintendo Switch OLED a...,[],"[Video Games, Nintendo Switch, Accessories, Ca...",29.99,"[-1, -1, -1, -1, -1, 811, 2540, 313, 4394, 1846]",1846,0.094859,0
4,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B0BLFYF8K2,4.0,1630263342566,1616,4127,Computers,"Logitech G600 MMO Gaming Mouse, RGB Backlit, 2...","[With 20 buttons, the Logitech G600 MMO Gaming...","[Video Games, PC, Accessories, Gaming Mice]",37.99,"[1459, 1860, 3264, 569, 2143, 773, 4483, 296, ...",4525,0.184153,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AFPSHZKKUL2YDGIDBQQUNRGE5MXQ,B07DD7QTBM,0.0,1641121394999,1672,2700,None,Just Dance 2019 - Xbox One Standard Edition,"[Dance to your own beat with Just Dance 2019, ...","[Video Games, Xbox One, Games]",12.3,"[-1, -1, -1, 2830, 1342, 1294, 1749, 2558, 385...",4560,0.259751,0
1894,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,49,1887,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 3179, 1489, 2225, 3399, 3142, 4247, 3801,...",3413,0.219673,1
1895,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,6619,2246,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 2780, 3158, 130, 1164, 1030]",1030,0.064468,1
1896,AEBTSECUK7ZEECNSRHQLMKO3E5VA,B002BSA388,0.0,1642567970979,18651,3724,Video Games,Super Mario Galaxy 2,"[Product Description, Launch into a new univer...","[Video Games, Legacy Systems, Nintendo Systems...",80.82,"[-1, -1, -1, -1, 2135, 2729, 14, 3639, 3118, 955]",955,0.095067,0


In [26]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

# Clean up

In [27]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/26 22:59:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run 005-content-based at: http://localhost:5003/#/experiments/3/runs/de7b347bd0b04a90bf5eb84dab62732f.
2024/09/26 22:59:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/3.
